In [1]:
import json
import pandas as pd
import glob
import os

In [2]:
df_old = pd.read_pickle('new_annotation/final/cvat_annotations_20240806_pandas.pkl')
df_old[(df_old["crop_name"]=='unknown_1907_crop_2468_1674.jpg') & (df_old['group_id'] == 1)]

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name
6481,"[[912.0, 429.08], [911.44, 439.28], [912.11, 4...",(mount,false,false,1,1,unknown_1907_crop_2468_1674.jpg
6482,"[[570.84, 312.72], [571.41, 304.97], [571.78, ...",hârat,false,false,1,1,unknown_1907_crop_2468_1674.jpg
6483,"[[914.84, 366.93], [914.6, 374.04], [913.93, 3...",sion,false,false,1,2,unknown_1907_crop_2468_1674.jpg
6484,"[[560.58, 252.28], [560.62, 255.11], [560.68, ...",al,false,false,1,2,unknown_1907_crop_2468_1674.jpg
6485,"[[920.47, 312.9], [919.55, 319.38], [918.74, 3...",str.),false,false,1,3,unknown_1907_crop_2468_1674.jpg
6486,"[[560.87, 211.8], [560.9, 218.04], [560.53, 22...",kalâ,false,false,1,3,unknown_1907_crop_2468_1674.jpg


In [3]:
df = None

for img_dir in glob.glob('new_annotation/maps_res/*'):
    final_pkl = glob.glob(img_dir + "\\*ultimate_corr*.pkl")
    assert len(final_pkl) == 1
    df_ = pd.read_pickle(final_pkl[0])
    df = pd.concat((df, df_))

In [4]:
df.loc[(df['group_id'] == 1) & (df['text'] == '(MOUNT'), 'group_id'] = 2
df.loc[(df['group_id'] == 1) & (df['text'] == 'SION'), 'group_id'] = 2
df.loc[(df['group_id'] == 1) & (df['text'] == 'STR.)'), 'group_id'] = 2

In [5]:
df_select = df[['vertices_final', "text_final", "illegible", "truncated", "group_id", "sequence_order", "crop_name"]]

In [6]:
df_renamed = df_select.rename(columns={'text_final': 'text', 'vertices_final': 'vertices'})

In [7]:
group_counts = df_renamed.groupby(['group_id', 'crop_name'])['group_id'].transform('count')

In [8]:
df_renamed.loc[group_counts == 1, 'group_id'] = -1

In [9]:
df_renamed.loc[df_renamed['group_id'] == -1, 'sequence_order'] = None

In [10]:
df_renamed[(df_renamed["crop_name"]=='unknown_1907_crop_2468_1674.jpg') & (df_renamed['group_id'] == 2)]

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name
23,"[[(913.7, 490.0), (912.0, 429.08), (914.1, 421...",(MOUNT,false,false,2,1,unknown_1907_crop_2468_1674.jpg
25,"[[(913.03, 414.89), (914.84, 366.93)], [(928.5...",SION,false,false,2,2,unknown_1907_crop_2468_1674.jpg
27,"[[(915.5, 358.51), (920.47, 312.9)], [(935.66,...",STR.),false,false,2,3,unknown_1907_crop_2468_1674.jpg


In [11]:
def check_single_word(df_):
    # Check if the group has only one row
    if len(df_) == 1:
        # Access the single value in 'group_id' correctly using .iloc[0]
        if df_['group_id'].iloc[0] != -1:
            display(df_)

# Apply the function to each group
df_renamed.groupby(['group_id', 'crop_name']).apply(check_single_word)

""


In [12]:
def check_multi_word(df_):
    # Check if the group has only one row
    if df_['group_id'].iloc[0] != -1:
        # Access the single value in 'group_id' correctly using .iloc[0]
        if len(df_) == 1:
            display(df_)

# Apply the function to each group
df_renamed.groupby(['group_id', 'crop_name']).apply(check_multi_word)

""


In [13]:
df_sorted = df_renamed.sort_values(by=['crop_name', 'group_id', 'sequence_order']).reset_index(drop=True)

In [14]:
df_sorted

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name
0,"[[(1981.629486084, 995.9494628906), (1982.2384...",R,false,false,-1,None,12148_btv1b53027745sf1_0.jpg
1,"[[(1953.5, 1316.43), (1997.99, 1344.34)], [(19...",AVE,false,false,-1,None,12148_btv1b53027745sf1_0.jpg
2,"[[(1907.7, 0.0), (1950.95, 53.37)], [(1929.25,...",ngand,false,false,-1,None,12148_btv1b53027745sf1_0.jpg
3,"[[(1055.73, 0.49), (1073.29, 20.91), (1085.83,...",refjulhe,false,false,-1,None,12148_btv1b53027745sf1_0.jpg
4,"[[(315.78, 406.17), (579.67, 404.41)], [(580.1...",NEUILLY,false,false,-1,None,12148_btv1b53027745sf1_0.jpg
...,...,...,...,...,...,...,...
7470,"[[(122.83, 859.32), (223.15, 860.86)], [(224.5...",Thor,false,false,6,2,wolff_1872_crop_887_1671.jpg
7471,"[[(538.2, 768.5), (618.35, 771.02)], [(617.07,...",Ruin.,false,false,7,1,wolff_1872_crop_887_1671.jpg
7472,"[[(619.62, 769.75), (650.15, 771.02)], [(648.8...",d.,false,false,7,2,wolff_1872_crop_887_1671.jpg
7473,"[[(540.0, 798.93), (677.05, 800.27)], [(675.31...",Johañiter,false,false,7,3,wolff_1872_crop_887_1671.jpg


In [15]:
def reorder_sequence(df_):
    if len(df_) > 1 and df_["group_id"].iloc[0] != -1:
        df_['sequence_order'] = range(1, len(df_) + 1)
    return df_

# Apply the function to reorder the sequence_order for each group
df_reordered = df_renamed.groupby(['group_id', 'crop_name'], group_keys=False).apply(reorder_sequence).reset_index(drop=True)

In [16]:
df_reordered[df_reordered["crop_name"]=="weller_1860_crop_752_605.jpg"]

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name
74,"[[(0.01, 894.19), (51.23, 823.61), (92.51, 778...",QUARTER,false,false,-1,None,weller_1860_crop_752_605.jpg
151,"[[(683.09, 583.16), (802.62, 582.97)], [(802.3...",Gethsemane,false,false,-1,None,weller_1860_crop_752_605.jpg
229,"[[(1.97, 586.66), (15.81, 560.13), (44.25, 522...",QUARTER,false,false,-1,None,weller_1860_crop_752_605.jpg
307,"[[(281.02, 542.61), (331.28, 531.91), (346.71,...",Barracks,false,false,-1,None,weller_1860_crop_752_605.jpg
393,"[[(720.47, 872.52), (774.78, 871.27)], [(774.0...",Tombs,false,false,2,1,weller_1860_crop_752_605.jpg
...,...,...,...,...,...,...,...
5552,"[[(519.52, 638.77), (547.82, 638.77)], [(547.2...",the,false,false,59,4,weller_1860_crop_752_605.jpg
5566,"[[(549.96, 636.37), (567.57, 637.44), (587.86,...",Eternal,false,false,59,5,weller_1860_crop_752_605.jpg
5580,"[[(613.81, 637.04), (653.85, 637.57)], [(652.2...",Gate,false,false,59,6,weller_1860_crop_752_605.jpg
5594,"[[(523.85, 652.52), (578.31, 654.12)], [(577.7...",Golden,false,false,59,7,weller_1860_crop_752_605.jpg


In [17]:
df_reordered.to_pickle('new_annotation/final/cvat_annotations_20240904_pandas.pkl')

In [18]:
all_crop_labels = df_reordered

In [19]:
all_crop_labels['geographic_area'] = all_crop_labels['crop_name'].apply(lambda x: 'paris' if x.split('_')[0] == 'BHVP' or x.split('_')[0].isdigit() else 'jerusalem')
all_crop_labels.geographic_area.value_counts()

geographic_area
paris        3950
jerusalem    3525
Name: count, dtype: int64

In [20]:
def region_count(region, all_crop_labels):
    all_crop_labels = all_crop_labels[all_crop_labels['illegible'] == 'false']
    mwtoponyms = all_crop_labels[all_crop_labels['group_id'] != -1][['crop_name', 'group_id', 'geographic_area']].drop_duplicates()
    swtoponyms = all_crop_labels[all_crop_labels['group_id'] == -1][['crop_name', 'group_id', 'geographic_area']]
    print(region + ' - legible bounding boxes: ', len(all_crop_labels[all_crop_labels['geographic_area'] == region]))
    print(region + ' - legible MW toponyms', len(mwtoponyms[mwtoponyms['geographic_area'] == region]))
    print(region + ' - legible SW toponyms', len(swtoponyms[swtoponyms['geographic_area'] == region]))
region_count('paris', all_crop_labels)
region_count('jerusalem', all_crop_labels)

paris - legible bounding boxes:  3788
paris - legible MW toponyms 1116
paris - legible SW toponyms 838
jerusalem - legible bounding boxes:  3417
jerusalem - legible MW toponyms 807
jerusalem - legible SW toponyms 908


In [21]:
no_text = all_crop_labels[~all_crop_labels['text'].notna()]
no_text[no_text['illegible'] == 'false']

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name,geographic_area


In [22]:
in_group = all_crop_labels[all_crop_labels['group_id'] != -1]
in_group[~in_group['sequence_order'].notna()]

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name,geographic_area


In [28]:
import json

dict_list = []
for image in all_crop_labels['crop_name'].drop_duplicates():

    # isolate labels w and wout groups
    tmp = all_crop_labels[all_crop_labels['crop_name'] == image].copy()
    tmp_single_word = tmp[tmp['group_id'] == -1]
    tmp_group = tmp[tmp['group_id'] != -1]

    # name image
    img_dict = {}
    img_dict['image'] = image

    # create list of toponyms
    groups_list = []
    for index, row in tmp_single_word.iterrows():
        groups_list.append([{'vertices': row['vertices'], 'text': row['text'], 'illegible': row['illegible'], 'truncated': row['truncated']}])
    for group_id in tmp_group['group_id'].drop_duplicates():
        tmp_group_id_lst = []
        tmp_group_id = tmp_group[tmp_group['group_id'] == group_id].copy()
        for index, row in tmp_group_id.iterrows():
            tmp_group_id_lst.append({'vertices': row['vertices'], 'text': row['text'], 'illegible': row['illegible'], 'truncated': row['truncated']})
        groups_list.append(tmp_group_id_lst)

    # add toponyms list (list of lists, where single-word toponyms are single element lists) to img_dict
    img_dict['groups'] = groups_list

    dict_list.append(img_dict)

with open('new_annotation/final/cvat_annotations_' + '20240904' + '_ICDARstructure.json', 'w') as f:
    json.dump(dict_list, f)

In [31]:
df_reordered[df_reordered["crop_name"]=="12148_btv1b8440780mf1_0.png"]

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name,geographic_area
22,"[[(445.4, 319.5), (755.3, 322.89), (982.2, 323...",MILITAIR,false,true,-1,None,12148_btv1b8440780mf1_0.png,paris
102,"[[(464.85, 451.57), (578.44, 452.23)], [(579.7...",par,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
182,"[[(103.29, 432.44), (182.93, 432.44)], [(183.8...",de,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
261,"[[(16.2663173676, 522.515411377), (18.88991546...",de,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
341,"[[(0.76, 432.33), (58.6, 433.44)], [(59.93, 49...",le,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
421,"[[(37.6, 879.66), (123.56, 851.82), (239.1, 82...",Seine,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
500,"[[(323.5045928955, 742.2063674927), (325.55422...",R.,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
578,"[[(604.81, 427.26), (684.52, 426.59)], [(683.8...",J.,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
654,"[[(694.49, 425.93), (776.79, 426.59)], [(780.1...",G.,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
730,"[[(790.81, 423.93), (982.78, 425.26)], [(984.1...",Heck,false,false,-1,None,12148_btv1b8440780mf1_0.png,paris
